In [132]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
titanic = pd.read_csv("datasets/titanic.csv")

predictors = titanic.drop(["survived", "age_was_missing"], axis=1).values
target = to_categorical(titanic.survived)

# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]
print(n_cols)
print(target[:5])

### Complie and fit the model with `loss='categorical_crossentropy'` for classification

In [ ]:
# Set up the model
model = Sequential()

# Add the first layer
model.add(Dense(32, activation = 'relu', input_shape = (n_cols,)))

# Add the output layer
model.add(Dense(2, activation= 'softmax'))

# Compile the model
model.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fit the model
model.fit(predictors, target, epochs=20)

# Calculate predictions: predictions
predictions = model.predict(predictors)

# Calculate predicted probability of survival: predicted_prob_true
predicted_prob_true = predictions[:,1]

# Print Accuracy
loss, accuracy = model.evaluate(predictors, target)
print('Accuracy:', accuracy)

## Save & Load

In [ ]:
model.save("Saved/titanic_model.h5")

my_model = load_model("Saved/titanic_model.h5")
my_model.summary()

pred = my_model.predict(predictors)

## Compare Models

In [ ]:
early_stopping = EarlyStopping(patience=3)

# Model 1
model1 = Sequential()

model1.add(Dense(10, activation = 'relu', input_shape = (n_cols,)))

model1.add(Dense(10, activation= 'relu'))

model1.add(Dense(2, activation= 'softmax'))

model1.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model1_training = model1.fit(predictors, target, epochs=15, validation_split=0.2, callbacks=[early_stopping], verbose=False)


# Model 2 same as model 1 but with more nodes
model2 = Sequential()

model2.add(Dense(100, activation = 'relu', input_shape = (n_cols,)))

model2.add(Dense(50, activation = 'relu'))

model2.add(Dense(2, activation= 'softmax'))

model2.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model2_training = model2.fit(predictors, target, epochs=15, validation_split=0.2, callbacks=[early_stopping], verbose=False)


# Model 3 same as model 1 but with more layers
model3 = Sequential()

model3.add(Dense(10, activation = 'relu', input_shape = (n_cols,)))

model3.add(Dense(10, activation = 'relu'))

model3.add(Dense(10, activation = 'relu'))

model3.add(Dense(2, activation= 'softmax'))

model3.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model3_training = model3.fit(predictors, target, epochs=15, validation_split=0.2, callbacks=[early_stopping], verbose=False)




print(model1_training.history['val_accuracy'])
print(model2_training.history['val_accuracy'])
print(model3_training.history['val_accuracy'])

print("final_accuracy1 = ", model1_training.history['val_accuracy'][-1])
print("final_accuracy2 = ", model2_training.history['val_accuracy'][-1])
print("final_accuracy3 = ", model3_training.history['val_accuracy'][-1])
 
# Plot the Comparasion
plt.plot(model1_training.history['val_accuracy'], 'r', model2_training.history['val_accuracy'], 'b', model3_training.history['val_accuracy'], 'g')
plt.xlabel('Epochs')
plt.xlim(0, 15)
plt.ylabel('Validation accuracy score')
plt.show()
